In [13]:
import pandas as pd
import numpy as np
import requests

In [14]:
colors = pd.read_csv('../Datas/colors.csv')
elements = pd.read_csv('../Datas/elements.csv')
inventory_parts = pd.read_csv('../Datas/inventory_parts.csv')
inventories = pd.read_csv('../Datas/inventories.csv')
sets = pd.read_csv('../Datas/sets.csv')
themes= pd.read_csv('../Datas/themes.csv')
part_categories = pd.read_csv('../Datas/part_categories.csv')
parts = pd.read_csv('../Datas/parts.csv')

### Joindre inventory_parts et inventories

In [15]:
#merge les fichiers pour ajouter l'infos set_num
inventory_add_set_num = pd.merge(inventory_parts[:-1], inventories[['set_num','id']], left_on='inventory_id', right_on='id', how='left').drop('id',axis=1).drop('img_url',axis=1)

#renommer quantity et set_num
inventory_add_set_num.rename(columns={'quantity':'part_quantity'},inplace=True)
inventory_add_set_num.rename(columns={'set_num':'set_id'},inplace = True)

#vérifier l'existence de pièces de remplassement pour voir si je garde la colonne
is_spare_true = inventory_add_set_num.loc[inventory_add_set_num['is_spare'] == 't']

### Adjoindre maintenant le df sets et cleaner le résultat

In [16]:
inventories_and_sets = pd.merge(inventory_add_set_num, sets, left_on ='set_id', right_on ='set_num', how='left').drop('set_num',axis=1).drop('img_url', axis=1)

columns_to_convert = ['year','theme_id','num_parts']
default_value = 0  # Valeur par défaut à utiliser pour remplacer les valeurs non valides
inventories_and_sets[columns_to_convert] = inventories_and_sets[columns_to_convert].fillna(default_value).astype(int)

#renommer les colonnes
inventories_and_sets.rename(columns={'name':'set_nom'},inplace = True)
inventories_and_sets.rename(columns = {'year':'set_annee'},inplace = True)
inventories_and_sets.rename(columns = {'part_num':'piece_id'},inplace = True)
inventories_and_sets.rename(columns = {'part_quantity':'piece_quantity'},inplace = True)
inventories_and_sets.rename(columns = {'num_parts':'set_nb_pieces'},inplace = True)

### Ajouter les infos sur les univers

In [17]:
inventories_sets_univers = pd.merge(inventories_and_sets, themes[['id','name']], left_on = 'theme_id', right_on = 'id', how='left').drop('id',axis=1)
inventories_sets_univers.rename(columns = {'theme_id':'univers_id'}, inplace = True)
inventories_sets_univers.rename(columns = {'name':'univers_nom'}, inplace = True)

#déplacer ma colonne 'num_parts'
inventories_sets_univers.insert(8, 'set_nb_pieces', inventories_sets_univers.pop('set_nb_pieces'))

#supprimer colonnes piece_quantity et inventory_id que je nutilise pas
inventories_sets_univers = inventories_sets_univers.drop('piece_quantity',axis=1)
sets_univers = inventories_sets_univers.drop('inventory_id', axis=1)


### Supprimer les lignes NaN et trier selon colonne 'piece_id'

In [18]:
sets_univers = sets_univers.dropna(subset=['set_nom'])
sets_univers = sets_univers.sort_values(by='set_annee', ascending=True)

### Vérifier les codes hexadécimaux dans le fichiers 'colors'

In [19]:
# Convertir la colonne en string
colors['rgb'] = colors['rgb'].astype(str)

colors_lenght = colors[colors['rgb'].str.len() == 6]

### Ajouter les colonnes 'couleur_nom' et 'couleur_code'

In [20]:
colors_df = pd.merge(sets_univers, colors, left_on = 'color_id', right_on = 'id', how = 'left').drop('id', axis=1)

In [21]:
colors_df.insert(4, 'couleur_nom', colors_df.pop('name'))
colors_df.insert(5, 'couleur_code', colors_df.pop('rgb'))
colors_df.insert(3, 'couleur_id', colors_df.pop('color_id'))
colors_df.insert(6, 'is_trans', colors_df.pop('is_trans'))
colors_df.insert(6, 'set_id', colors_df.pop('set_id'))

### Transformer ma colonne couleur_code en hexadécimal

In [22]:
colors_df['couleur_code'] = '#' + colors_df['couleur_code']

### Supprimer les sets 'unused'

In [23]:
colors_df = colors_df[~colors_df['set_nom'].str.contains('Unused')]

### Créer le nouveau fichier csv couleurs

In [24]:
colors_df.to_csv('../Datas/colors_cleaned.csv', index = False)

In [26]:
display(colors_df.head(3))

,piece_id,is_spare,couleur_id,couleur_nom,couleur_code,is_trans,set_id,set_nom,set_annee,set_nb_pieces,univers_id,univers_nom
0,brickslot0004,f,15,White,#FFFFFF,f,700.3-1,Medium Gift Set (ABB),1949,142,365,System
1,brickslot0002,f,4,Red,#C91A09,f,700.3-1,Medium Gift Set (ABB),1949,142,365,System
2,brickslot0002,f,2,Green,#237841,f,700.3-1,Medium Gift Set (ABB),1949,142,365,System
